In [12]:
!pip uninstall -y fsspec

Found existing installation: fsspec 2023.6.0
Uninstalling fsspec-2023.6.0:
  Successfully uninstalled fsspec-2023.6.0


In [13]:

!pip install fsspec==2023.6.0

  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2023.6.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-runtime-cu1

In [2]:
# Step 1.1: Install required libraries (run only once)
!pip install transformers datasets sentencepiece -q
!pip install scikit-learn matplotlib -q

# Step 1.2: Check if GPU is available
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [5]:

# --------------------------
# Install dependencies
# --------------------------
!pip install transformers datasets sentencepiece -q
!pip install scikit-learn matplotlib nltk -q
# --------------------------
# Imports and GPU check
# --------------------------
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import spearmanr
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.corpus import wordnet
import random
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# --------------------------
# Load STS-B dataset
# --------------------------
dataset = load_dataset("glue", "stsb")
df = dataset["train"].to_pandas()
positive_pairs = df[df["label"] >= 4.0][["sentence1", "sentence2"]].reset_index(drop=True)
negative_pairs = df[df["label"] <= 2.0][["sentence1", "sentence2"]].reset_index(drop=True)
print(f"Positive pairs: {len(positive_pairs)}, Negative pairs: {len(negative_pairs)}")

# --------------------------
# Data Augmentation function
# --------------------------
def synonym_replacement(sentence, n=1):
    words = sentence.split()
    new_words = words.copy()
    candidates = list(set([word for word in words if wordnet.synsets(word)]))
    if not candidates:
        return sentence
    count = 0
    while count < n:
        word = random.choice(candidates)
        synonyms = wordnet.synsets(word)
        if synonyms:
            syn_words = synonyms[0].lemma_names()
            if syn_words:
                synonym = syn_words[0].replace("_", " ")
                new_words = [synonym if w == word else w for w in new_words]
                count += 1
    return " ".join(new_words)

# --------------------------
# Create augmented pairs
# --------------------------
N = 1000
positive_samples = positive_pairs.sample(N, random_state=42)
negative_samples = negative_pairs.sample(N, random_state=42)
positive_samples["label"] = 1
negative_samples["label"] = 0

augmented_sentences = []
for _, row in positive_samples.iterrows():
    aug_s1 = synonym_replacement(row["sentence1"], n=1)
    augmented_sentences.append({"sentence1": aug_s1, "sentence2": row["sentence2"], "label": 1})

aug_df = pd.DataFrame(augmented_sentences)
positive_aug = pd.concat([positive_samples, aug_df]).reset_index(drop=True)
combined_pairs = pd.concat([positive_aug, negative_samples]).sample(frac=1).reset_index(drop=True)
print(f"Total pairs after augmentation: {len(combined_pairs)}")

# --------------------------
# Tokenizer
# --------------------------
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# --------------------------
# Dataset and DataLoader
# --------------------------
class ContrastivePairDataset(Dataset):
    def __init__(self, df):
        self.sentence1 = list(df["sentence1"])
        self.sentence2 = list(df["sentence2"])

    def __len__(self):
        return len(self.sentence1)

    def __getitem__(self, idx):
        return self.sentence1[idx], self.sentence2[idx]

aug_dataset = ContrastivePairDataset(combined_pairs)
aug_dataloader = DataLoader(aug_dataset, batch_size=32, shuffle=True)

# --------------------------
# Contrastive Model
# --------------------------
class ContrastiveModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super(ContrastiveModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_embeddings = output.last_hidden_state[:, 0]
        return cls_embeddings

def info_nce_loss(emb1, emb2, temperature=0.05):
    emb1 = F.normalize(emb1, dim=1)
    emb2 = F.normalize(emb2, dim=1)
    sim_matrix = torch.matmul(emb1, emb2.T) / temperature
    batch_size = emb1.size(0)
    labels = torch.arange(batch_size).to(emb1.device)
    loss = F.cross_entropy(sim_matrix, labels)
    return loss

# --------------------------
# Vanilla BERT baseline
# --------------------------
class VanillaBERTModel(nn.Module):
    def __init__(self, model_name="bert-base-uncased"):
        super(VanillaBERTModel, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
    def forward(self, input_ids, attention_mask, token_type_ids):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        return output.last_hidden_state[:, 0]

# Evaluate vanilla BERT
sts_val = load_dataset("glue", "stsb", split="validation")
df_val = sts_val.to_pandas()[["sentence1", "sentence2", "label"]].dropna().reset_index(drop=True)
vanilla_model = VanillaBERTModel().to(device)
vanilla_model.eval()

batch_size = 64
cos_sims_base = []
for i in range(0, len(df_val), batch_size):
    batch_s1 = df_val["sentence1"][i:i+batch_size].tolist()
    batch_s2 = df_val["sentence2"][i:i+batch_size].tolist()
    enc1 = tokenizer(batch_s1, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    enc2 = tokenizer(batch_s2, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    with torch.no_grad():
        emb1 = vanilla_model(enc1["input_ids"], enc1["attention_mask"], enc1["token_type_ids"])
        emb2 = vanilla_model(enc2["input_ids"], enc2["attention_mask"], enc2["token_type_ids"])
    sim = cosine_similarity(emb1.cpu(), emb2.cpu()).diagonal()
    cos_sims_base.extend(sim)

true_scores = df_val["label"].values / 5.0
pred_scores_base = np.array(cos_sims_base)
corr_base, _ = spearmanr(true_scores, pred_scores_base)
print(f"🟢 Vanilla BERT Spearman Correlation: {corr_base:.4f}")

# --------------------------
# Train contrastive model
# --------------------------
model = ContrastiveModel().to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

for epoch in range(3):
    total_loss = 0
    model.train()
    for batch in aug_dataloader:
        sent1, sent2 = batch
        enc1 = tokenizer(list(sent1), padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
        enc2 = tokenizer(list(sent2), padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
        emb1 = model(enc1["input_ids"], enc1["attention_mask"], enc1["token_type_ids"])
        emb2 = model(enc2["input_ids"], enc2["attention_mask"], enc2["token_type_ids"])
        loss = info_nce_loss(emb1, emb2)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        total_loss += loss.item()
    avg_loss = total_loss / len(aug_dataloader)
    print(f"Epoch {epoch+1}: Avg InfoNCE Loss = {avg_loss:.4f}")

# --------------------------
# Evaluate trained model
# --------------------------
model.eval()
cos_sims = []
for i in range(0, len(df_val), batch_size):
    batch_s1 = df_val["sentence1"][i:i+batch_size].tolist()
    batch_s2 = df_val["sentence2"][i:i+batch_size].tolist()
    enc1 = tokenizer(batch_s1, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    enc2 = tokenizer(batch_s2, padding=True, truncation=True, return_tensors="pt", max_length=128).to(device)
    with torch.no_grad():
        emb1 = model(enc1["input_ids"], enc1["attention_mask"], enc1["token_type_ids"])
        emb2 = model(enc2["input_ids"], enc2["attention_mask"], enc2["token_type_ids"])
    sim = cosine_similarity(emb1.cpu(), emb2.cpu()).diagonal()
    cos_sims.extend(sim)

pred_scores = np.array(cos_sims)
corr, _ = spearmanr(true_scores, pred_scores)
print(f"🔍 Spearman Correlation after contrastive training: {corr:.4f}")

model_path = "/content/saved_model"
os.makedirs(model_path, exist_ok=True)

tokenizer.save_pretrained(model_path)
model.bert.save_pretrained(model_path)
print("✅ Model saved to:", model_path)



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Using device: cuda
Positive pairs: 1406, Negative pairs: 1967
Total pairs after augmentation: 3000
🟢 Vanilla BERT Spearman Correlation: 0.3174
Epoch 1: Avg InfoNCE Loss = 0.5631
Epoch 2: Avg InfoNCE Loss = 0.2905
Epoch 3: Avg InfoNCE Loss = 0.1966
🔍 Spearman Correlation after contrastive training: 0.7600
✅ Model saved to: /content/saved_model
